# Assignment 2
## Create a Neural Network using PyTorch for FashionMNIST dataset

In [1]:
%matplotlib inline



# NN with PyTorch

PyTorch has two primitives to work with data <https://pytorch.org/docs/stable/data.html>: 
- ``torch.utils.data.DataLoader`` and 
- ``torch.utils.data.Dataset``.

``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.




### Import the necessary libraries

In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

PyTorch offers domain-specific libraries such as 
- `TorchText <https://pytorch.org/text/stable/index.html>`_, 
- `TorchVision <https://pytorch.org/vision/stable/index.html>`_, and 
- `TorchAudio <https://pytorch.org/audio/stable/index.html>`_, 
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like 
CIFAR, COCO (`full list here <https://pytorch.org/docs/stable/torchvision/datasets.html>`_). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



### Download training data from open datasets

In [4]:
training_data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())

### Download test data from open datasets

In [5]:
test_data = datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element 
in the dataloader iterable will return a batch of 64 features and labels.



In [6]:
batch_size = 64

# Create data loaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size, shuffle=True)

--------------




Creating Models
------------------
To define a neural network in PyTorch, we create a class that inherits 
from `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_. 

We define the layers of the network in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. 

To accelerate operations in the neural network, we move it to the GPU if available.



### Get cpu or gpu device for training

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


### Define model
- Flatten the image
- Sequential Layers
    - Linear layer
    - ReLU layer

In [9]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [10]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


--------------




Optimizing the Model Parameters
----------------------------------------
To train a model, we need
- `loss function` <https://pytorch.org/docs/stable/nn.html#loss-functions>
- `optimizer` <https://pytorch.org/docs/stable/optim.html> 



In [11]:
loss_fn = nn.CrossEntropyLoss()

In [12]:
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and 
backpropagates the prediction error to adjust the model's parameters. 



In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.



In [19]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns 
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.188160  [    0/60000]
loss: 2.264143  [ 6400/60000]
loss: 2.180099  [12800/60000]
loss: 2.190869  [19200/60000]
loss: 2.138578  [25600/60000]
loss: 2.217078  [32000/60000]
loss: 2.220592  [38400/60000]
loss: 2.274604  [44800/60000]
loss: 2.092018  [51200/60000]
loss: 2.227077  [57600/60000]
Test Error: 
 Accuracy: 28.8%, Avg loss: 0.033909 

Epoch 2
-------------------------------
loss: 2.144667  [    0/60000]
loss: 2.224989  [ 6400/60000]
loss: 2.117445  [12800/60000]
loss: 2.145448  [19200/60000]
loss: 2.090295  [25600/60000]
loss: 2.139733  [32000/60000]
loss: 2.178716  [38400/60000]
loss: 1.989480  [44800/60000]
loss: 2.142176  [51200/60000]
loss: 2.007169  [57600/60000]
Test Error: 
 Accuracy: 28.9%, Avg loss: 0.032791 

Epoch 3
-------------------------------
loss: 1.984472  [    0/60000]
loss: 1.971917  [ 6400/60000]
loss: 2.137500  [12800/60000]
loss: 2.169159  [19200/60000]
loss: 2.050408  [25600/60000]
loss: 2.133822  [32000/600

--------------




Saving Models
-------------
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [21]:
torch.save(model, 'themodel.pth')

Loading Models
----------------------------

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it. 



In [23]:
themodel = torch.load('themodel.pth')

This model can now be used to make predictions.



In [25]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sandal", Actual: "Ankle boot"
